<a href="https://colab.research.google.com/github/tirolab/FDOPA-PET-analysis/blob/main/Dopaminergic_denervation_FDOPA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import scipy
import seaborn as sns
import tqdm
from scipy import stats
import sys
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# Models 1, 2, 3

In [ ]:
# Importing data and defining the feature list
data =
label =

INFO_patient = [x for x in list(data.columns) if x.split('_')[0]=='INFO']

FEATURES = 
LABEL = 

In [ ]:
# Data cleanup
striatum_D = data.drop_duplicates(subset='INFO_PatientID', keep="first").reset_index()[FEATURES]
striatum_G = data.drop_duplicates(subset='INFO_PatientID', keep="last").reset_index()[FEATURES]
radiomics_features = striatum_D + striatum_G
radiomics_features = radiomics_features/2
patients_info = data.drop_duplicates(subset='INFO_PatientID', keep="first").reset_index()[INFO_patient]
datawithlabel = pd.concat((radiomics_features, label), axis=1)
patients_info = patients_info[patients_info.INFO_ActualFrameDuration=='6.0 min']
datawithlabel = datawithlabel.loc[patients_info.index]

In [ ]:
# Model code (replacing the feature list as appropriate)
scores = []
coeffs = []
for repet in tqdm.tqdm(range(1000)):
  X_train, X_test, y_train, y_test = train_test_split(datawithlabel[FEATURES], datawithlabel[LABEL].values.flatten(), test_size=0.25)
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  cv = LogisticRegressionCV(cv = 5, penalty = 'l1', solver = 'saga', scoring='roc_auc', Cs = np.logspace(-1,1,20), max_iter=10000)
  cv.fit(X_train, y_train)
  scores.append(cv.score(X_test, y_test)*100)
  coeffs.append(list(cv.coef_.T.flatten()))
print(np.mean(scores), np.std(scores))

In [ ]:
# List of scores
scores.sort()
scores_df = DataFrame(scores,columns=['Score'])
scores_df.to_excel("bootstrap scores.xlsx")

In [ ]:
# List of features
proba_df = pd.DataFrame(data = (abs(np.array(coeffs))>0).sum(0)/len(coeffs), index=FEATURES, columns=['proba'])
scores_df = pd.DataFrame(data = np.array(coeffs).sum(0)/(np.array(coeffs)!=0).sum(0), index=FEATURES, columns=['score'])
scores_df = scores_df.fillna(0)
dataframe = pd.concat([proba_df,scores_df],1).sort_values('proba', ascending=False)
dataframe.to_excel("feature list.xlsx") 

# External validation

In [ ]:
data_val = 
label_val =
FINAL_FEATURES = 
# Training
X_train, y_train = datawithlabel[FINAL_FEATURES], datawithlabel[LABEL].values.flatten()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
logreg = LogisticRegression(penalty = 'none')
logreg.fit(X_train, y_train)
# Testing
auroc = logreg.score(scaler.transform(data_val[FINAL_FEATURES]), label_val.values.flatten())*100

tn, fp, fn, tp = confusion_matrix(label_val.values.flatten(), logreg.predict(scaler.transform(data_val[FINAL_FEATURES]))).ravel()
sensitivity = tp / (tp+fn) * 100
specificity = tn / (tn+fp) * 100
accuracy = (tp+tn)/(tp+tn+fp+fn) * 100

print("AUROC: {} \t Acc: {} \t Sensitivity: {} \t Specificity : {}".format(auroc, accuracy, sensitivity, specificity))

# Model for clinical scores

In [ ]:
scores = 
score_label = 
mean_imputation = scores.fillna(scores.median())
scores = []
coeffs = []
for repet in tqdm.tqdm(range(1000)):
  X_train, X_test, y_train, y_test = train_test_split(mean_imputation, score_label.values.flatten(), test_size=0.25)
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  cv = LogisticRegressionCV(cv = 5, penalty = 'l1', solver = 'saga', scoring='roc_auc', Cs = np.logspace(-1,1,20), max_iter=10000)
  cv.fit(X_train, y_train)
  scores.append(cv.score(X_test, y_test)*100)
  coeffs.append(list(cv.coef_.T.flatten()))
print(np.mean(scores), np.std(scores))

# CCC

In [ ]:
# CCC code from https://github.com/stylianos-kampakis/supervisedPCA-Python/blob/master/Untitled.py
def concordance_correlation_coefficient(y_true, y_pred,
                       sample_weight=None,
                       multioutput='uniform_average'):
    cor=np.corrcoef(y_true,y_pred)[0][1]
    mean_true=np.mean(y_true)
    mean_pred=np.mean(y_pred)
    var_true=np.var(y_true)
    var_pred=np.var(y_pred)
    sd_true=np.std(y_true)
    sd_pred=np.std(y_pred)
    numerator=2*cor*sd_true*sd_pred
    denominator=var_true+var_pred+(mean_true-mean_pred)**2
    return numerator/denominator

# Pearson correlogram

In [ ]:
path =
listfile = os.listdir(path)
list_corr = []
for name_file in listfile:
  data = pd.read_excel(path+name_file, header=0)
  striatum_D = data.drop_duplicates(subset='INFO_PatientID', keep="first").reset_index()[usefuldata]
  striatum_G = data.drop_duplicates(subset='INFO_PatientID', keep="last").reset_index()[usefuldata]
  radiomics_features = striatum_D + striatum_G
  radiomics_features.drop(RIM, axis=1, inplace=True)
  radiomics_features = radiomics_features/2
  X = radiomics_features[FEATURES]
  X.rename(columns={"CONVENTIONAL_TLG(mL)(onlyForPETorNM)": "CONVENTIONAL_TLG(mL)",
                    'SHAPE_Sphericity(onlyFor3DROI))':'SHAPE_Sphericity',
                    'SHAPE_Surface(mm2)(onlyFor3DROI)':'SHAPE_Surface(mm2)',
                    'SHAPE_Compacity(onlyFor3DROI)':'SHAPE_Compacity'}, inplace=True)
  list_corr.append(X.corr('pearson'))
ax = sns.clustermap(sum(list_corr)/len(list_corr))